In [ ]:
# This notebook was used to create my dimension tables from the original Sakila data. Code here is based on my midterm project. 

In [38]:
# Importing necessary tools
import os
import json
import numpy
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

import numpy
import datetime
import certifi
import pymongo

In [39]:
# MySQL access creds (replace generics)
mysql_args = {
    "uid" : "nav3up",
    "pwd" : "Min78Gan&Parle",
    "hostname" : "nav3up-mysql.mysql.database.azure.com",
    "dbname" : "sakila_dw"
}

In [40]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [41]:
src_dbname = "sakila"
dst_dbname = "sakila_dw"
host_name = "nav3up-mysql.mysql.database.azure.com"
user_id = "nav3up"
pwd = "Min78Gan&Parle"

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

In [42]:
sql_rental = "SELECT * FROM sakila.rental;"
df_rental = get_dataframe(user_id, pwd, host_name, src_dbname, sql_rental)
df_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


In [43]:
sql_payment = "SELECT * FROM sakila.payment;"
df_payment = get_dataframe(user_id, pwd, host_name, src_dbname, sql_payment)
df_payment.head(2)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30


In [44]:
sql_customer = "SELECT * FROM sakila.customer;"
df_customer = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customer)
df_customer.head(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [45]:
sql_inventory = "SELECT * FROM sakila.inventory;"
df_inventory = get_dataframe(user_id, pwd, host_name, src_dbname, sql_inventory)
df_inventory.head(2)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


In [46]:
sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name, src_dbname, sql_film)
df_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [47]:
# Making the dimension tables proper (adding keys)

df_rental.insert(0, "rental_key", range(1, df_rental.shape[0]+1))
df_payment.insert(0, "payment_key", range(1, df_payment.shape[0]+1))
df_customer.insert(0, "customer_key", range(1, df_customer.shape[0]+1))
df_inventory.insert(0, "inventory_key", range(1, df_inventory.shape[0]+1))
df_film.insert(0, "film_key", range(1, df_film.shape[0]+1))

In [48]:
db_operation = "insert"

tables = [('dim_rental', df_rental, 'rental_key'),
          ('dim_payment', df_payment, 'payment_key'),
          ('dim_customer', df_customer, 'customer_key'),
         ('dim_inventory', df_inventory, 'inventory_key'),
         ('dim_film', df_film, 'film_key')]

In [72]:
df_film.head()

,film_key,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [56]:
# Insert all into the new sakila_dw
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
# https://drupal.stackexchange.com/questions/316995/multiple-primary-key-defined-when-importing-db-to-dev

In [145]:
# Creating the fact_rentals table
df_fact_rentals = pd.merge(df_rental, df_inventory, on='inventory_id', how='inner')
df_fact_rentals.head()

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,inventory_key,film_id,store_id,last_update_y
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,367,80,1,2006-02-15 05:09:17
1,1576,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,2006-02-15 21:30:53,367,80,1,2006-02-15 05:09:17
2,3582,3584,2005-07-06 04:16:43,367,207,2005-07-13 07:08:43,1,2006-02-15 21:30:53,367,80,1,2006-02-15 05:09:17
3,10503,10507,2005-08-01 11:22:20,367,45,2005-08-04 13:18:20,2,2006-02-15 21:30:53,367,80,1,2006-02-15 05:09:17
4,13637,13641,2005-08-20 07:34:42,367,281,2005-08-26 05:18:42,1,2006-02-15 21:30:53,367,80,1,2006-02-15 05:09:17


In [146]:
# Don't really need these at the moment
df_fact_rentals.drop(['last_update_x','last_update_y'], axis=1, inplace=True)

In [147]:
df_fact_rentals.head(2)

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,inventory_key,film_id,store_id
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,367,80,1
1,1576,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,367,80,1


In [148]:
# Adding in the film table
df_fact_rentals = pd.merge(df_fact_rentals, df_film, on='film_id', how='inner')
df_fact_rentals.head()

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,inventory_key,film_id,store_id,...,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42
1,1576,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42
2,3582,3584,2005-07-06 04:16:43,367,207,2005-07-13 07:08:43,1,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42
3,10503,10507,2005-08-01 11:22:20,367,45,2005-08-04 13:18:20,2,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42
4,13637,13641,2005-08-20 07:34:42,367,281,2005-08-26 05:18:42,1,367,80,1,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42


In [149]:
# Preparing df_payment to be added
#df_payment.drop(['customer_id','staff_id','last_update'], axis = 1, inplace = True)
df_payment.head(2)

,payment_key,payment_id,rental_id,amount,payment_date
0,1,1,76,2.99,2005-05-25 11:30:37
1,2,2,573,0.99,2005-05-28 10:35:23


In [150]:
df_fact_rentals = pd.merge(df_fact_rentals, df_payment, on='rental_id', how='inner')
df_fact_rentals.head(2)

,rental_key,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,inventory_key,film_id,store_id,...,rental_rate,length,replacement_cost,rating,special_features,last_update,payment_key,payment_id,amount,payment_date
0,1,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,367,80,1,...,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42,3503,3504,2.99,2005-05-24 22:53:30
1,1576,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,367,80,1,...,2.99,148,21.99,G,Trailers,2006-02-15 05:03:42,8826,8828,3.99,2005-06-16 04:03:28


In [151]:
df_fact_rentals.drop(['last_update'], axis=1, inplace=True)

In [152]:
# Preparing df_customer to be added
#df_customer.rename(columns={"first_name":"renter_first_name"}, inplace=True)
#df_customer.rename(columns={"last_name":"renter_last_name"}, inplace=True)
#df_customer.drop(['address_id','create_date','last_update','email','store_id'], axis=1,inplace=True)
df_customer.head(3)

,customer_key,customer_id,renter_first_name,renter_last_name,active
0,1,1,MARY,SMITH,1
1,2,2,PATRICIA,JOHNSON,1
2,3,3,LINDA,WILLIAMS,1


In [153]:
# Adding df_customer now
df_fact_rentals = pd.merge(df_fact_rentals, df_customer, on="customer_id", how="inner")

In [154]:
# removing unecessary columns
df_fact_rentals.drop(['rental_id','inventory_id','customer_id','staff_id','film_id','store_id','language_id','original_language_id','special_features'], axis=1, inplace = True)
df_fact_rentals.head(2)

,rental_key,rental_date,return_date,inventory_key,film_key,title,description,release_year,rental_duration,rental_rate,...,replacement_cost,rating,payment_key,payment_id,amount,payment_date,customer_key,renter_first_name,renter_last_name,active
0,1,2005-05-24 22:53:30,2005-05-26 22:04:30,367,80,BLANKET BEVERLY,A Emotional Documentary of a Student And a Gir...,2006,7,2.99,...,21.99,G,3503,3504,2.99,2005-05-24 22:53:30,130,CHARLOTTE,HUNTER,1
1,4483,2005-07-08 01:07:54,2005-07-16 03:19:54,2614,573,MICROCOSMOS PARADISE,A Touching Character Study of a Boat And a Stu...,2006,6,2.99,...,22.99,PG-13,3512,3513,4.99,2005-07-08 01:07:54,130,CHARLOTTE,HUNTER,1


In [155]:
# As above
df_fact_rentals.drop(['title','description','release_year','rating','payment_id','amount'], axis=1, inplace=True)
df_fact_rentals.drop(['length','replacement_cost'], axis=1, inplace=True)
df_fact_rentals.head(2)

,rental_key,rental_date,return_date,inventory_key,film_key,rental_duration,rental_rate,payment_key,payment_date,customer_key,renter_first_name,renter_last_name,active
0,1,2005-05-24 22:53:30,2005-05-26 22:04:30,367,80,7,2.99,3503,2005-05-24 22:53:30,130,CHARLOTTE,HUNTER,1
1,4483,2005-07-08 01:07:54,2005-07-16 03:19:54,2614,573,6,2.99,3512,2005-07-08 01:07:54,130,CHARLOTTE,HUNTER,1


In [156]:
sql_date = "SELECT * FROM sakila_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_date)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20050101,2005-01-01,2005/01/01,01/01/2005,01/01/2005,7,Saturday,1,1,Weekend,...,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3
1,20050102,2005-01-02,2005/01/02,01/02/2005,02/01/2005,1,Sunday,2,2,Weekend,...,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3


In [157]:
# Integrating the date dimension table
df_dim_rental_date = df_dim_date.rename(columns={"date_key" : "rental_date_key", "full_date" : "rental_date"})
df_fact_rentals.rental_date = df_fact_rentals.rental_date.astype('datetime64[ns]').dt.date

df_fact_rentals = pd.merge(df_fact_rentals, df_dim_rental_date, on='rental_date', how='left')
df_fact_rentals.drop(['rental_date'], axis=1, inplace=True)
df_fact_rentals.head(2)

,rental_key,return_date,inventory_key,film_key,rental_duration,rental_rate,payment_key,payment_date,customer_key,renter_first_name,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,2005-05-26 22:04:30,367,80,7,2.99,3503,2005-05-24 22:53:30,130,CHARLOTTE,...,N,2,2005,2005-05,2005Q2,11,4,2005,2005-11,2005Q4
1,4483,2005-07-16 03:19:54,2614,573,6,2.99,3512,2005-07-08 01:07:54,130,CHARLOTTE,...,N,3,2005,2005-07,2005Q3,1,1,2006,2006-01,2006Q1


In [158]:
df_fact_rentals.drop(['date_name', 'date_name_us', 'date_name_eu',
       'day_of_week', 'day_name_of_week', 'day_of_month', 'day_of_year',
       'weekday_weekend', 'week_of_year', 'month_name', 'month_of_year',
       'is_last_day_of_month', 'calendar_quarter', 'calendar_year',
       'calendar_year_month', 'calendar_year_qtr', 'fiscal_month_of_year',
       'fiscal_quarter', 'fiscal_year', 'fiscal_year_month',
       'fiscal_year_qtr'], axis=1, inplace=True)

In [159]:
# Now for return_date
df_dim_return_date = df_dim_date.rename(columns={"date_key" : "return_date_key", "full_date" : "return_date"})
df_fact_rentals.return_date = df_fact_rentals.return_date.astype('datetime64[ns]').dt.date

df_fact_rentals = pd.merge(df_fact_rentals, df_dim_return_date, on='return_date', how='left')
df_fact_rentals.drop(['return_date'], axis=1, inplace=True)
df_fact_rentals.drop(['date_name', 'date_name_us', 'date_name_eu',
       'day_of_week', 'day_name_of_week', 'day_of_month', 'day_of_year',
       'weekday_weekend', 'week_of_year', 'month_name', 'month_of_year',
       'is_last_day_of_month', 'calendar_quarter', 'calendar_year',
       'calendar_year_month', 'calendar_year_qtr', 'fiscal_month_of_year',
       'fiscal_quarter', 'fiscal_year', 'fiscal_year_month',
       'fiscal_year_qtr'], axis=1, inplace=True)
df_fact_rentals.head(2)

,rental_key,inventory_key,film_key,rental_duration,rental_rate,payment_key,payment_date,customer_key,renter_first_name,renter_last_name,active,rental_date_key,return_date_key
0,1,367,80,7,2.99,3503,2005-05-24 22:53:30,130,CHARLOTTE,HUNTER,1,20050524,20050526.0
1,4483,2614,573,6,2.99,3512,2005-07-08 01:07:54,130,CHARLOTTE,HUNTER,1,20050708,20050716.0


In [160]:
# And payment date 
df_dim_payment_date = df_dim_date.rename(columns={"date_key" : "payment_date_key", "full_date" : "payment_date"})
df_fact_rentals.payment_date = df_fact_rentals.payment_date.astype('datetime64[ns]').dt.date

df_fact_rentals = pd.merge(df_fact_rentals, df_dim_payment_date, on='payment_date', how='left')
df_fact_rentals.drop(['payment_date'], axis=1, inplace=True)
df_fact_rentals.drop(['date_name', 'date_name_us', 'date_name_eu',
       'day_of_week', 'day_name_of_week', 'day_of_month', 'day_of_year',
       'weekday_weekend', 'week_of_year', 'month_name', 'month_of_year',
       'is_last_day_of_month', 'calendar_quarter', 'calendar_year',
       'calendar_year_month', 'calendar_year_qtr', 'fiscal_month_of_year',
       'fiscal_quarter', 'fiscal_year', 'fiscal_year_month',
       'fiscal_year_qtr'], axis=1, inplace=True)
df_fact_rentals.head(2)

,rental_key,inventory_key,film_key,rental_duration,rental_rate,payment_key,customer_key,renter_first_name,renter_last_name,active,rental_date_key,return_date_key,payment_date_key
0,1,367,80,7,2.99,3503,130,CHARLOTTE,HUNTER,1,20050524,20050526.0,20050524
1,4483,2614,573,6,2.99,3512,130,CHARLOTTE,HUNTER,1,20050708,20050716.0,20050708


In [161]:
# Inserting the main key
df_fact_rentals.insert(0, "fact_rental_key", range(1, df_fact_rentals.shape[0]+1))

In [163]:
# Load the final fact table into mySQL...
table_name = "fact_rentals"
primary_key = "fact_rental_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_rentals, table_name, primary_key, db_operation)